In [ ]:
DESCRIPTION = """Using quantum espresso to compute the stresses associated with different strain states and fitting the elastic tensor."""

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
from pymatgen.io.pwscf import PWInput
from pymatgen.core import Structure
from pymatgen.analysis.elasticity import *
from pymatgen.io.vasp.inputs import *
from pymatgen.core.tensors import symmetry_reduce
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import os

from simtool import getValidatedInputs, DB, findInstalledSimToolNotebooks, searchForSimTool
import nanohubremote as nr

In [ ]:
%load_ext yamlmagic

In [ ]:
%%yaml INPUTS

mp_id:
    type: Text
    description: Materials Project ID for chosen structure
        
squid_id:
    type: Text
    description: Simulation ID for relaxed structure from CellRelaxDFT
        
strains:
    type: List
    description: List of strain magnitudes
        
shears:
    type: List
    description: List of shear magnitudes

In [ ]:
defaultInputs = getValidatedInputs(INPUTS)
if defaultInputs:
    globals().update(defaultInputs)

In [ ]:
EXTRA_FILES = ['pseudo']

In [ ]:
%%yaml OUTPUTS

external_pressure:
    type: Number
    description: External pressure of the prestine structure queried from CellRelaxDFT
    units: gigapascal

strain_matrices:
    type: Array
    description: Array of strain matrices that were actually used in the generation of the deformed structures and fitting of the elastic tensor (only different from the input if sym_red is True)
        
stress_tensors:
    type: Array
    description: Array of stress tensors as arrays that were computed by quantum espresso for each deformed structure and used in the fit for the elastic tensor

deformed_structures:
    type: List
    description: List of pymatgen Structure objects in dictionary format that were generated by the applying strains to the pristine structure
        
energies:
    type: List
    description: List of energies per formula unit from the DFT calculations on each deformed structure
    
elastic_tensor:
    type: Array
    description: Elastic tensor in voigt notation fit with pymatgen using the Moore-Penrose pseudo-inverse method
        
C11:
    type: Number
    description: Elastic constant C11
    units: gigapascal
        
C12:
    type: Number
    description: Elastic constant C12
    units: gigapascal
        
C13:
    type: Number
    description: Elastic constant C13
    units: gigapascal
        
C14:
    type: Number
    description: Elastic constant C14
    units: gigapascal
        
C15:
    type: Number
    description: Elastic constant C15
    units: gigapascal
        
C16:
    type: Number
    description: Elastic constant C16
    units: gigapascal
        
C22:
    type: Number
    description: Elastic constant C22
    units: gigapascal
        
C23:
    type: Number
    description: Elastic constant C23
    units: gigapascal
        
C24:
    type: Number
    description: Elastic constant C24
    units: gigapascal
        
C25:
    type: Number
    description: Elastic constant C25
    units: gigapascal
        
C26:
    type: Number
    description: Elastic constant C26
    units: gigapascal
        
C33:
    type: Number
    description: Elastic constant C33
    units: gigapascal
        
C34:
    type: Number
    description: Elastic constant C34
    units: gigapascal
        
C35:
    type: Number
    description: Elastic constant C35
    units: gigapascal
        
C36:
    type: Number
    description: Elastic constant C36
    units: gigapascal

C44:
    type: Number
    description: Elastic constant C44
    units: gigapascal
        
C45:
    type: Number
    description: Elastic constant C45
    units: gigapascal
        
C46:
    type: Number
    description: Elastic constant C46
    units: gigapascal
        
C55:
    type: Number
    description: Elastic constant C55
    units: gigapascal
        
C56:
    type: Number
    description: Elastic constant C56
    units: gigapascal
        
C66:
    type: Number
    description: Elastic constant C66
    units: gigapascal
        
bulk_modulus_reuss:
    type: Number
    description: Bulk modulus derived from the elastic tensor using the Reuss method (constant strain)
    units: gigapascal
        
bulk_modulus_voigt:
    type: Number
    description: Bulk modulus derived from the elastic tensor using the Voigt method (constant stress)
    units: gigapascal
        
bulk_modulus_vrh:
    type: Number
    description: Bulk modulus derived from the elastic tensor using the Voigt-Reuss-Hill method (average of Reuss and Voigt)
    units: gigapascal
        
shear_modulus_reuss:
    type: Number
    description: Shear modulus derived from the elastic tensor using the Reuss method (constant strain)
    units: gigapascal
        
shear_modulus_voigt:
    type: Number
    description: Shear modulus derived from the elastic tensor using the Voigt method (constant stress)
    units: gigapascal
        
shear_modulus_vrh:
    type: Number
    description: Shear modulus derived from the elastic tensor using the Voigt-Reuss-Hill method (average of Reuss and Voigt)
    units: gigapascal
        
homogeneous_poisson:
    type: Number
    description: Homogeneous poisson ratio derived from the elastic tensor
        
youngs_modulus:
    type: Number
    description: Youngs modulus derived from the elastic tensor
    units: gigapascal
        
calc_params:
    type: Dict
    description: Dictionary of various DFT parameters

In [ ]:
auth_data = { 'grant_type' : 'tool' }
with open(os.environ["SESSIONDIR"]+"/resources") as file:
    lines = [line.split(" ", 1) for line in file.readlines()]
    properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
    auth_data["sessiontoken"] = properties["session_token"]
    auth_data["sessionnum"] = properties["sessionid"]
    
session = nr.Session(auth_data, url='https://nanohub.org/api')

In [ ]:
tool = 'cellrelaxdft'

revisions = [0]

data_list = {}

for revision in revisions:
    search = {
            'tool':tool,
            'filters':json.dumps([
                                  {'field':'squid','operation':'==','value':squid_id},
                                ]),
            'results':json.dumps([
                                'input.mp_id',
                                'input.pseudo',
                                'input.vdw_corr',
                                'input.KE_cutoff',
                                'input.i_steps',
                                'input.e_steps',
                                'input.scf_conv',
                                'input.energy_conv',
                                'input.force_conv',
                                'input.spin_polar',
                                'input.is_metal',
                                'input.hubbard_U_on',
                                'input.hubbard_projector',
                                'input.hubbard_U_values',
                                'input.hubbard_U_orbitals',
                                'output.pressure',
                                'output.structure_dict'
                                ]),    
            'limit':10000,    
            'revision':revision,
            'simtool' : 1
             }

    req_json = session.requestPost('results/dbexplorer/search', data=search, timeout=60)
    req_json = req_json.json()
    data_list[f'{revision}'] = pd.DataFrame(req_json['results'])


data = pd.concat([data_list[f'{revision}'] for revision in revisions], axis=0, ignore_index=True)
data

In [ ]:
mp_id_cellrelaxdft = data['input.mp_id'][0]
if mp_id != mp_id_cellrelaxdft:
    raise ValueError(f'{mp_id} is not the same as what is retreived using squid: {mp_id_cellrelaxdft}')

In [ ]:
struct = Structure.from_dict(data['output.structure_dict'][0], fmt='cif')
sga = SpacegroupAnalyzer(struct)
struct = sga.get_conventional_standard_structure()
n_atoms = struct.num_sites
n_atom_types = len(struct.elements)
is_metal = data['input.is_metal'][0]
ext_press = data['output.pressure'][0]

In [ ]:
if is_metal:
    k_x = round((1/struct.lattice.abc[0])*(2*np.pi)/0.16)
    k_y = round((1/struct.lattice.abc[1])*(2*np.pi)/0.16)
    k_z = round((1/struct.lattice.abc[2])*(2*np.pi)/0.16)
else:
    k_x = round((1/struct.lattice.abc[0])*(2*np.pi)/0.22)
    k_y = round((1/struct.lattice.abc[1])*(2*np.pi)/0.22)
    k_z = round((1/struct.lattice.abc[2])*(2*np.pi)/0.22)
    
KE_cutoff = data['input.KE_cutoff'][0]

In [ ]:
i_steps = data['input.i_steps'][0]
e_steps = data['input.e_steps'][0]
scf_conv = data['input.scf_conv'][0]
energy_conv = data['input.energy_conv'][0]
force_conv = data['input.force_conv'][0]
vdw_corr = data['input.vdw_corr'][0]
pseudo = data['input.pseudo'][0]
spin_polar = data['input.spin_polar'][0]
hubbard_U_on = data['input.hubbard_U_on'][0]
if hubbard_U_on:
    hubbard_projector = data['input.hubbard_projector'][0]
    hubbard_U_values = data['input.hubbard_U_values'][0]
    hubbard_U_orbitals = data['input.hubbard_U_orbitals'][0]
else:
    hubbard_projector = None
    hubbard_U_values = None
    hubbard_U_orbitals = None

In [ ]:
calc_params = {'pseudopotential':pseudo, 'vdw_correction':vdw_corr, 'kmesh':[k_x,k_y,k_z], 'kinetic_energy_cutoff':KE_cutoff, 'ionic_steps':i_steps, 'electronic_steps':e_steps, 
               'scf_convergence_criteria':scf_conv, 'energy_convergence_criteria':energy_conv, 'force_convergence_criteria':force_conv, 
               'spin_polarization':spin_polar, 'Hubbard_U':hubbard_U_on, 'Hubbard_projector':hubbard_projector, 'Hubbard_U_values':hubbard_U_values, 'Hubbard_U_orbital':hubbard_U_orbitals}

In [ ]:
def make_sim(name,struct,sym,**kwargs):
    """
    Generate quantum espresso input files using pymatgen's PWInput class
    
    Inputs:
        name: chosen name for your simulation (i.e. ionic_relax)
        struct: pymatgen structure object 
    Outputs: 
        n/a
    **kwargs:
        dictionaries to input to pymatgen's PWInput object
    """
    # Prepare dict of pseudopotentials (i.e. {'Mg': 'Mg.upf', 'O': 'O.upf'})
    elements = np.unique([site.species.elements[0].symbol for site in struct.sites])
    pseudo_dict = dict(zip(elements,[f"{element}.upf" for element in elements]))

    # Define input set
    input_set = PWInput(structure=struct,
                        pseudo=pseudo_dict,
                        **kwargs) # dictionaries corresponding to blocks in QE input files

    input_set.write_file(filename=f'{name}.in')
    # if we want to impose symmetry we remove the CELL_PARAMETERS card from qe in file
    if sym != 0:      
        with open(f'{name}.in') as f1:
            lines = f1.readlines()

        with open('tmp.in', 'w') as f2:
            f2.writelines(lines[:-4])
            
        os.rename('tmp.in', f'{name}.in')
    
def run_sim(name,struct,pseudo):
    """
    Submit quantum espresso runs to HPC clusters on nanoHUB
    
    Inputs:
        name: chosen name for your simulation (i.e. ionic_relax)
        struct: pymatgen structure object 
    Outputs: 
        n/a
    """
    # Write input and output files
    input_file = open(f'{name}.in','a')
    input_file.close()

    output_file = open(f'{name}.out', 'w')
    output_file.close()
    
    # Set up commands and files
    elements = np.unique([site.species.elements[0].symbol for site in struct.sites])
    pseudo_arg = "".join([f"-i ./pseudo/pseudo_{pseudo}_PBEsol/{element}.upf " for element in elements])
    COMMAND = f"espresso-7.1_pw > {output_file.name}"
    
    # Run simulation (1 node, 64 cpus, 24 hour walltime)
    !submit -n 64 -w '8:00:00' --noquota -e QE_DISABLE_GGA_PBE=0 --runName {name} {pseudo_arg} {COMMAND} -i {input_file.name}  

# Define helper functions for QE    
def get_qe_outputs_relax(file):
    """
    Extract outputs (energies, forces, structures) from qe .stdout files
    
    inputs:
        file: path to the file we want to extract outputs from
    outputs:
        dict: dictionary of the extracted outputs
    """
    
    output = open(file, "r")
    lines = output.readlines()
    iE = [] # energy at each ionic step, Ry
    eE = [[]] # energy at each electronic step, Ry
    P = [] # pressure, kbar
    F = [] # total force, Ry/au
    stresses = [] # stress tensor, kbar
    structures = [] # pymatgen structure objects, angstrom

    # Check for certain tags on lines, add variables to lists
    for i,line in enumerate(lines):
        if 'total energy' in line and '!' not in line and 'The' not in line:
            eE[-1].append(float(line.split()[3]))
        elif 'total energy' in line and '!' in line:
            eE.append([])
            iE.append(float(line.split()[4]))
        elif 'P=' in line:
            P.append(float(line.split()[5]))
            stresses.append(np.array([lines[i+1].split()[3:6],lines[i+2].split()[3:6],lines[i+3].split()[3:6]]).astype(float))
        elif "Total force" in line:
            F.append(float(line.split()[3]))
        elif 'CELL_PARAMETERS' in line:
            try:
                if 'alat' in line:
                    scale = float(line.split()[-1].split(')')[0])*0.529177
                else:
                    scale = 1.0
                lattice = scale*np.array([lines[i+1].split(),lines[i+2].split(),lines[i+3].split()]).astype(float)
                sites = []
                atoms = []
                j=6
                while ("End" not in lines[i+j].strip()) and (lines[i+j].strip()!=""):
                    sites.append(np.array(lines[i+j].split()[1:]).astype(float))
                    atoms.append(lines[i+j].split()[0])
                    j=j+1
                lattice_obj = Lattice(lattice)
                print(lattice, atoms, sites)
                test_struct = Structure(lattice,atoms,sites)
                structures.append(test_struct)
            except:
                pass
    eE = eE[:-1]

    # return output dictionary
    return {'ionic_energies':iE,'electronic_energies':eE,'pressures':P,'forces':F,'stresses':stresses,'structures':structures}

In [ ]:
electron_dict = {'electron_maxstep':e_steps,'conv_thr':scf_conv}

if is_metal:
    system_dict = {'ecutwfc':int(KE_cutoff),'occupations':'smearing','smearing':'gauss','degauss':0.02,'vdw_corr':vdw_corr}
else:
    system_dict = {'ecutwfc':int(KE_cutoff),'vdw_corr':vdw_corr}
    
if spin_polar:
    electron_dict['mixing_beta']=0.3
    system_dict['nspin']=2
    for i in range(n_atom_types):
        system_dict['starting_magnetization('+str(i+1)+')']=0.6
        if not is_metal:
            system_dict['occupations']='smearing'
            system_dict['smearing']='gauss'
            system_dict['degauss']=0.02
            
def add_hubbard(name,struct,hubbard_projector,hubbard_U_values):
    h_lines = ['HUBBARD {'+hubbard_projector+'} \n']
    elements = np.unique([site.species.elements[0].symbol for site in struct.sites])
    for e in elements:
        u = hubbard_U_values[e]
        o = hubbard_U_orbitals[e] 
        h_lines.append(f'U {e}-{o} {u}\n')
            
    f = open(f'{name}.in','r')
    lines = f.readlines()
    for line in h_lines:
        lines.append(line)
    f_new = open('tmp.in','w')
    f_new.writelines(lines)
    f.close()
    f_new.close()
    
    os.rename('tmp.in', f'{name}.in')
    
if hubbard_U_on:
    electron_dict['mixing_beta']=0.3

In [ ]:
# Create pymatgen Strain objects from matrices
# # Generate strain matrices
strain_matrices = []
for ind in [(0, 0), (1, 1), (2, 2)]:
    for amount in strains:
        strain = Strain.from_index_amount(ind, amount)
        strain_matrices.append(strain)
        
for ind in [(0, 1), (0, 2), (1, 2)]:
    for amount in shears:
        strain = Strain.from_index_amount(ind, amount)
        strain_matrices.append(strain)

# Generate deformed structures
deformations = []

for s_mat in strain_matrices:
    deform_matrix = s_mat.get_deformation_matrix()
    deformations.append(deform_matrix)
    
strain_arrays = [np.array(s) for s in strain_matrices]

# if sym_red:
#     sym_dict = symmetry_reduce(deformations, struct)
#     deformations = list(sym_dict)

#     strain_mats_reduced = [Strain.from_deformation(defo) for defo in deformations]
#     strain_mats = strain_mats_reduced
    
deform_structs = [defo.apply_to_structure(struct) for defo in deformations]
deform_struct_dicts = [defo_struct.as_dict() for defo_struct in deform_structs]
    
# Calculate each stress tensor
system_dict = {'ecutwfc':KE_cutoff,'vdw_corr':vdw_corr}
electron_dict = {'electron_maxstep':e_steps,'conv_thr':scf_conv}

stress_tensors = []
stress_arrays = []
energies = []
for i,d_struct in enumerate(deform_structs):
    print(f'Running calculation on deformed structure {i} out of {len(deform_structs)}')

    make_sim("relax", d_struct, sym=0,
             control={'pseudo_dir':'./',
                      'calculation':'relax',
                      'outdir':'./',
                      'tstress':True,
                      'nstep':i_steps,
                      'etot_conv_thr':energy_conv,
                      'forc_conv_thr':force_conv,
                      'disk_io':'nowf'},
             system=system_dict,
             electrons=electron_dict,
             kpoints_grid=[k_x,k_y,k_z])
    
    # Add Hubbard U correction if needed
    if hubbard_U_on:
        add_hubbard("relax", struct, hubbard_projector, hubbard_U_values)

    run_sim("relax",d_struct,pseudo)
    
    try:
        relax_dict = get_qe_outputs_relax('relax.stdout')
    except:
        print('Getting Relax Dictionary Failed')
        
    s_tensor = -Stress(relax_dict['stresses'][-1]/10)  # convert kbar to GPa
    stress_tensors.append(s_tensor)
    stress_arrays.append(np.array(s_tensor))
    
    energies.append(relax_dict['ionic_energies'][-1]*13.605684958731/struct.composition.get_integer_formula_and_factor()[1])
    
    os.mkdir(f'strain_{i}-{len(deform_structs)}')
    os.system(f'mv relax* strain_{i}-{len(deform_structs)}')
    os.system('rm -r pw*')

In [ ]:
# Fit elastic tensor
elastic_tensor = ElasticTensor.from_pseudoinverse(strains=strain_matrices,stresses=stress_tensors)

In [ ]:
# Compute mechanical properties
# # Bulk modulus
k_reuss = elastic_tensor.k_reuss
k_voigt = elastic_tensor.k_voigt
k_vrh = elastic_tensor.k_vrh
# # Shear modulus
g_reuss = elastic_tensor.g_reuss
g_voigt = elastic_tensor.g_voigt
g_vrh = elastic_tensor.g_vrh
# # Homogeneous poisson
poisson = elastic_tensor.homogeneous_poisson
# # Young's modulus
y_mod = elastic_tensor.y_mod*1e-9

In [ ]:
db = DB(OUTPUTS)

In [ ]:
db.save('external_pressure',ext_press)
db.save('strain_matrices',np.array(strain_arrays))
db.save('stress_tensors', np.array(stress_arrays))
db.save('deformed_structures', deform_struct_dicts)
db.save('energies', energies)
db.save('elastic_tensor', elastic_tensor.voigt)
db.save('C11',elastic_tensor.voigt[0][0])
db.save('C12',elastic_tensor.voigt[1][0])
db.save('C13',elastic_tensor.voigt[2][0])
db.save('C14',elastic_tensor.voigt[3][0])
db.save('C15',elastic_tensor.voigt[4][0])
db.save('C16',elastic_tensor.voigt[5][0])
db.save('C22',elastic_tensor.voigt[1][1])
db.save('C23',elastic_tensor.voigt[2][1])
db.save('C24',elastic_tensor.voigt[3][1])
db.save('C25',elastic_tensor.voigt[4][1])
db.save('C26',elastic_tensor.voigt[5][1])
db.save('C33',elastic_tensor.voigt[2][2])
db.save('C34',elastic_tensor.voigt[3][2])
db.save('C35',elastic_tensor.voigt[4][2])
db.save('C36',elastic_tensor.voigt[5][2])
db.save('C44',elastic_tensor.voigt[3][3])
db.save('C45',elastic_tensor.voigt[4][3])
db.save('C46',elastic_tensor.voigt[5][3])
db.save('C55',elastic_tensor.voigt[4][4])
db.save('C56',elastic_tensor.voigt[5][4])
db.save('C66',elastic_tensor.voigt[5][5])
db.save('bulk_modulus_reuss',k_reuss)
db.save('bulk_modulus_voigt',k_voigt)
db.save('bulk_modulus_vrh',k_vrh)
db.save('shear_modulus_reuss',g_reuss)
db.save('shear_modulus_voigt',g_voigt)
db.save('shear_modulus_vrh',g_vrh)
db.save('homogeneous_poisson',poisson)
db.save('youngs_modulus',y_mod)
db.save('calc_params',calc_params)